In [1]:
import dspy

In [2]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

/home/ec2-user/miniconda3/envs/dspy/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


(20, 50)

In [4]:
dataset.train[0].items()

[('question',
  'At My Window was released by which American singer-songwriter?'),
 ('answer', 'John Townes Van Zandt')]

# Use Ollama
- Use the `Ollamalocal` in the dspy to interact with ollama model.

In [13]:
ollama_model = dspy.OllamaLocal(
    model='phi3',
    model_type='text',
    max_tokens=350,
    temperature=0.7,
    top_p=0.9,
    frequency_penalty=1.17,
    top_k=40
)

In [14]:
ollama_model("Tell me about the weather on pluto?")

[" Pluto, being a dwarf planet located in our solar system's Kuiper Belt, has an extremely cold and harsh environment. Its atmosphere is very thin compared to Earth's and consists mainly of nitrogen (N2), with traces of methane (CH4) and carbon monoxide (CO).\n\nThe surface temperature on Pluto averages around -375 to -400 degrees Fahrenheit (-225 to -240 degrees Celsius). It's essential to note that the conditions vary significantly between daytime and nighttime, with temperatures ranging from about 40 Kelvin (minus 231.8°F or minus 146.7°C) during midday summer at Pluto's equator to around -350 K (-238 °C/-152 °F) in winter nighttime across the entire planet due to its elongated orbit and axial tilt of about 120 degrees (similar to Uranus).\n\nPluto experiences extreme seasonal changes, with a long summer on one hemisphere lasting for half of Pluto's year – roughly 14 years. This extended period results in significant temperature variations across the planet over time and between day

# Configure LLM 
- In order to use the ollama model, set the DsPy settings.

In [15]:
dspy.settings.configure(lm=ollama_model)

- The `Signature` is more like a `Task` that you want to be performed.
- The docstring is like `system` prompt.
- For the example below (QA) even the input and the output fields are defined.

In [16]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words.")

In [17]:
# Define predictor. 
# The predictor is "informed" about the task to perform
generate_answer = dspy.Predict(BasicQA)

In [33]:
dev_example = devset[18]
pred = generate_answer(question=dev_example.question)
print(dev_example)
print(pred)

Example({'question': 'What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?', 'answer': 'English', 'gold_titles': {'Restaurant: Impossible', 'Robert Irvine'}}) (input_keys={'question'})
Prediction(
    answer='Answer: American citizen'
)


In [34]:
# You can inspect the ollama history to see the exact prompt
ollama_model.inspect_history(1)




Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words.

---

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Answer: Answer: American citizen





'\n\n\nAnswer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: often between 1 and 5 words.\n\n---\n\nQuestion: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?\nAnswer:\x1b Answer: American citizen\x1b\n\n\n'

# COT
- Lets check how this looks as a `chain of thought` task.
- Note DsPy changes the prompt template (at least thats how I see it)

In [35]:
# The predictor is changing, the signature is not
generate_answer_with_cot = dspy.ChainOfThought(BasicQA)

# Lets run it on the same input
pred = generate_answer_with_cot(question=dev_example.question)

# Print the prediction
print(pred)

Prediction(
    rationale='Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?\nReasoning: To find out the answer, we should identify who the main person associated with "Restaurant: Impossible" is. Once identified, research or look up their biography to determine their origin country as it relates to nationality.',
    answer='American'
)


# Using the Retrieval Logic

In [36]:
# A retrieval machine is needed
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(lm=ollama_model, rm=colbertv2_wiki17_abstracts)



In [40]:
# retrieve the top 2 passages/contexts
retrieve = dspy.Retrieve(k=3)
topk_passages = retrieve(dev_example.question).passages

print(f"Top {retrieve.k} passages for the question : {dev_example.question} \n", '-'*30, '\n')

for idx, passage in enumerate(topk_passages):
    print(f"{idx+1}]", passage, '\n')

Top 3 passages for the question : What is the nationality of the chef and restaurateur featured in Restaurant: Impossible? 
 ------------------------------ 

1] Restaurant: Impossible | Restaurant: Impossible is an American reality television series, featuring chef and restaurateur Robert Irvine, that aired on Food Network from 2011 to 2016. 

2] Jean Joho | Jean Joho is a French-American chef and restaurateur. He is chef/proprietor of Everest in Chicago (founded in 1986), Paris Club Bistro & Bar and Studio Paris in Chicago, The Eiffel Tower Restaurant in Las Vegas, and Brasserie JO in Boston. 

3] List of Restaurant: Impossible episodes | This is the list of the episodes for the American cooking and reality television series "Restaurant Impossible", produced by Food Network. The premise of the series is that within two days and on a budget of $10,000, celebrity chef Robert Irvine renovates a failing American restaurant with the goal of helping to restore it to profitability and promin

In [38]:
retrieve("When was the first FIFA World Cup held?").passages[0]

'History of the FIFA World Cup | The FIFA World Cup was first held in 1930, when FIFA president Jules Rimet decided to stage an international football tournament. The inaugural edition, held in 1930, was contested as a final tournament of only thirteen teams invited by the organization. Since then, the World Cup has experienced successive expansions and format remodeling to its current 32-team final tournament preceded by a two-year qualifying process, involving over 200 teams from around the world.'

# Program 1

- A complete program
- A RAG pipeline.
- Given a question, search and retrieve the top 3 passages in wikipedia and then use them as context for LLM
- Generate an answer from the LLM.

In [41]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words.")

# now the actual program. `Module`
- It needs 2 things.
    1. The `__init__` method that declares the sub-module it needs. In this case the `dspy.Retrieve` and `dspy.ChainOfThought`.
    2. The `forward` method will describe the control flow of answering the question using the modules we have.

In [46]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)



# Compiling the RAG
- A training set of 20 QAs
- A metric of validations, to check if the answer is correct. We can also check of the retrieved context is correct.
- A teleprompter that can optimize the programs.

To me this is more like having a `few shot examples`, and way to validate the responses from the LLM or the retrieval engine, and a teleprompter that optimzes your prompt for the task.

The author of DsPy are using a different language, but underneath the goal is just creating and optimizing a RAG prompt (basically prompt-engineering)

In [47]:
import dspy.evaluate
from dspy.teleprompt import BootstrapFewShot

# Validation logic: Check that the predict answer is correct
# Also check that the retrieved ccontext does actually contain the answer.

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM, answer_PM

# Set up a basic teleprompter, which will compile out RAG pipeline
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile
compile_rag = teleprompter.compile(RAG(), trainset=trainset)
    

 20%|██        | 4/20 [00:04<00:18,  1.15s/it]


In [48]:
# Now it compiled. Lets test it

In [49]:
test_question = "What castle did David Gregory inherit?"

pred = compile_rag(test_question)

In [51]:
print(f"Question: {test_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contets (truncated): {[c[:200]+'...' for c in pred.context]}")

Question: What castle did David Gregory inherit?
Predicted Answer: Kinnairdy Castle was inherited by David Gregory. Answer often between 1 and 5 words.
Retrieved Contets (truncated): ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'Gregory Tarchaneiotes | Gregory Tarchaneiotes (Greek: Γρηγόριος Ταρχανειώτης , Italian: "Gregorio Tracanioto" or "Tracamoto" ) was a "protospatharius" and the long-reigning catepan of Italy from 998 t...', 'David Gregory (mathematician) | David Gregory (originally spelt Gregorie) FRS (? 1659 – 10 October 1708) was a Scottish mathematician and astronomer. He was professor of mathematics at the University ...']


In [52]:
ollama_model.inspect_history(1)




Answer questions with short factoid answers.

---

Question: Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?
Answer: Tae Kwon Do Times

Question: "Everything Has Changed" is a song from an album released under which record label ?
Answer: Big Machine Records

Question: Which American actress who made their film debut in the 1995 teen drama "Kids" was the co-founder of Voto Latino?
Answer: Rosario Dawson

Question: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?
Answer: Aleem Sarwar Dar

Question: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?
Answer: "Outfield of Dreams"

Question: Who is older, Aleksandr Danilovich Aleksandrov or Anatoly Fomenko?
Answer: Aleksandr Danilovich Aleksandrov

Question: The Organisation that allows a community to influence their ope

'\n\n\nAnswer questions with short factoid answers.\n\n---\n\nQuestion: Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?\nAnswer: Tae Kwon Do Times\n\nQuestion: "Everything Has Changed" is a song from an album released under which record label ?\nAnswer: Big Machine Records\n\nQuestion: Which American actress who made their film debut in the 1995 teen drama "Kids" was the co-founder of Voto Latino?\nAnswer: Rosario Dawson\n\nQuestion: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?\nAnswer: Aleem Sarwar Dar\n\nQuestion: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?\nAnswer: "Outfield of Dreams"\n\nQuestion: Who is older, Aleksandr Danilovich Aleksandrov or Anatoly Fomenko?\nAnswer: Aleksandr Danilovich Aleksandrov\n\nQuestion: The Organisation that allows a commun